In [1]:
import workflow.dicom_workflow as workflow

In [2]:
workflow.get_directory()

reading total...50/1010 patient folders
reading total...100/1010 patient folders
reading total...150/1010 patient folders
reading total...200/1010 patient folders
reading total...250/1010 patient folders
reading total...300/1010 patient folders
reading total...350/1010 patient folders
reading total...400/1010 patient folders
reading total...450/1010 patient folders
reading total...500/1010 patient folders
reading total...550/1010 patient folders
reading total...600/1010 patient folders
reading total...650/1010 patient folders
reading total...700/1010 patient folders
reading total...750/1010 patient folders
reading total...800/1010 patient folders
reading total...850/1010 patient folders
reading total...900/1010 patient folders
reading total...950/1010 patient folders
reading total...1000/1010 patient folders
reading total...1010/1010 patient folders


In [3]:
workflow.dicom_directory

['C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-30178/3000566-03192/',
 'C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0008/01-01-2000-30141/3000549-21954/',
 'C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0009/01-01-2000-07045/3000538-29210/',
 'C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0013/01-01-2000-54750/3000551-80786/',
 'C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0015/01-01-2000-11146/3000610-28223/',
 'C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0016/01-01-2000-34278/3000580-73630/',
 'C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0019/01-01-2000-70623/3000523-06585/',
 'C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0020/01-01-2000-29935/3000525-47372/',
 'C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0023/01-01-2000-44207/3000628-26588/',
 'C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0024/01-01-2000-34802/3000557-80114/',
 'C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0026/01-01-2000-02665/3000519-40086/',
 'C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0027/01-01-2000-45559/3000614-12456/',
 'C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0028/01-01-2000-60855/3000563-52671/',

In [4]:
workflow.get_prepair_data()

Worker 1: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-30178/3000566-03192/
Worker 2: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0008/01-01-2000-30141/3000549-21954/
Worker 3: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0009/01-01-2000-07045/3000538-29210/
Worker 4: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0013/01-01-2000-54750/3000551-80786/
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-30178/3000566-03192/069.xml
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0008/01-01-2000-30141/3000549-21954/082.xml
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0009/01-01-2000-07045/3000538-29210/085.xml
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0013/01-01-2000-54750/3000551-80786/092.xml
reading total...50/256 patient DICOMs
reading total...50/125 patient DICOMs
reading total...50/133 patient DICOMs
reading total...50/133 patient DICOMs
reading total...100/125 patient DICOMs
reading total...100/133 patient DICOMs
reading total...100/256 patient DICOMs
reading total...100/133 patient DICOMs
reading total...125/125 patient DICOMs
W

C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0059/01-01-2000-67090/3000564-48912/190.xml
reading total...297/297 patient DICOMs
Worker 3: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0064/01-01-2000-71210/3000612-48666/
reading total...250/272 patient DICOMs
reading total...50/245 patient DICOMs
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0064/01-01-2000-71210/3000612-48666/199.xml
reading total...272/272 patient DICOMs
Worker 1: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0065/01-01-2000-03808/3000536-68085/
reading total...50/133 patient DICOMs
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0065/01-01-2000-03808/3000536-68085/201.xml
reading total...100/245 patient DICOMs
reading total...50/133 patient DICOMs
reading total...100/133 patient DICOMs
reading total...50/237 patient DICOMs
reading total...150/245 patient DICOMs
reading total...100/133 patient DICOMs
reading total...133/133 patient DICOMs
Worker 2: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0067/01-01-2000-71158/3000708-21600/
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0067/01-01-

reading total...50/310 patient DICOMs
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0157/01-01-2000-14060/3000697-12235/070.xml
reading total...100/139 patient DICOMs
reading total...50/139 patient DICOMs
reading total...50/133 patient DICOMs
reading total...100/310 patient DICOMs
reading total...139/139 patient DICOMs
Worker 1: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0158/01-01-2000-49056/3000737-58706/
reading total...100/139 patient DICOMs
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0158/01-01-2000-49056/3000737-58706/071.xml
reading total...100/133 patient DICOMs
reading total...150/310 patient DICOMs
reading total...139/139 patient DICOMs
Worker 3: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0200/01-01-2000-CAP LYM PREPOST-54643/6-Recon 3 C-A-P-08613/
reading total...133/133 patient DICOMs
Worker 4: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0203/01-01-2000-CHABDPELVIS-34578/4-Recon 2 C-A-P-15124/
reading total...50/251 patient DICOMs
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0200/01-01-2000-CAP LYM PREPOST-54643/6-Recon 3

reading total...450/471 patient DICOMs
reading total...500/513 patient DICOMs
reading total...50/564 patient DICOMs
reading total...471/471 patient DICOMs
reading total...513/513 patient DICOMs
Worker 4: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0311/01-01-2000-CT GUIDED LUNG B-49190/2-51615/
Worker 1: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0312/01-01-2000-CT CHEST WO CONTRAST-47558/2-12111/
reading total...100/264 patient DICOMs
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0311/01-01-2000-CT GUIDED LUNG B-49190/2-51615/050.xml
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0312/01-01-2000-CT CHEST WO CONTRAST-47558/2-12111/051.xml
reading total...100/564 patient DICOMs
reading total...50/225 patient DICOMs
reading total...50/250 patient DICOMs
reading total...150/264 patient DICOMs
reading total...150/564 patient DICOMs
reading total...100/225 patient DICOMs
reading total...100/250 patient DICOMs
reading total...200/264 patient DICOMs
reading total...200/564 patient DICOMs
reading total...150/225 patient DICOMs
r

reading total...350/421 patient DICOMs
reading total...246/246 patient DICOMs
Worker 4: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0348/01-01-2000-15484/30086-65896/
reading total...126/126 patient DICOMs
Worker 2: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0350/01-01-2000-CT LUNG SCREEN-63412/88402/
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0348/01-01-2000-15484/30086-65896/089.xml
reading total...100/130 patient DICOMs
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0350/01-01-2000-CT LUNG SCREEN-63412/88402/091.xml
reading total...400/421 patient DICOMs
reading total...130/130 patient DICOMs
Worker 3: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0353/01-01-2000-CT GUIDED LUNG B-02047/2-23235/
reading total...421/421 patient DICOMs
Worker 1: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0357/01-01-2000-CT LUNG SCREEN-10448/62394/
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0353/01-01-2000-CT GUIDED LUNG B-02047/2-23235/094.xml
reading total...50/158 patient DICOMs
reading total...50/270 patient DICOMs
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0357

...

reading total...400/484 patient DICOMs
Worker 1: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0862/01-01-2000-CT LUNG SCREEN-29911/40572/
reading total...200/417 patient DICOMs
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0862/01-01-2000-CT LUNG SCREEN-29911/40572/243.xml
reading total...436/436 patient DICOMs
Worker 2: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0865/01-01-2000-CT THORAX WCONTRAST-81073/3-10561/
reading total...450/484 patient DICOMs
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0865/01-01-2000-CT THORAX WCONTRAST-81073/3-10561/240.xml
reading total...250/417 patient DICOMs
reading total...50/132 patient DICOMs
reading total...484/484 patient DICOMs
Worker 4: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0866/01-01-2000-CT CHEST O CONTR-96082/80826/
reading total...50/141 patient DICOMs
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0866/01-01-2000-CT CHEST O CONTR-96082/80826/239.xml
reading total...300/417 patient DICOMs
reading total...100/132 patient DICOMs
reading total...132/132 patient DICOMs
Worker 1: C:/ntpu_projec

reading total...150/252 patient DICOMs
reading total...200/230 patient DICOMs
reading total...50/468 patient DICOMs
reading total...100/513 patient DICOMs
reading total...230/230 patient DICOMs
Worker 2: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0899/01-01-2000-46991/2-94709/
reading total...200/252 patient DICOMs
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0899/01-01-2000-46991/2-94709/206.xml
reading total...100/468 patient DICOMs
reading total...150/513 patient DICOMs
reading total...250/252 patient DICOMs
reading total...252/252 patient DICOMs
Worker 1: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0900/01-01-2000-CHEST-09248/32250-Recon 2 ACRIN LARGE-76100/
reading total...50/103 patient DICOMs
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0900/01-01-2000-CHEST-09248/32250-Recon 2 ACRIN LARGE-76100/205.xml
reading total...150/468 patient DICOMs
reading total...200/513 patient DICOMs
reading total...100/103 patient DICOMs
reading total...103/103 patient DICOMs
Worker 2: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0902/01-0

reading total...200/230 patient DICOMs
reading total...400/413 patient DICOMs
reading total...200/259 patient DICOMs
reading total...450/487 patient DICOMs
reading total...230/230 patient DICOMs
reading total...413/413 patient DICOMs
Worker 1: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0930/01-01-2000-CT LUNG SCREEN-20302/2-65646/
Worker 3: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0934/01-01-2000-CT LUNG SCREEN-69179/75647/
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0930/01-01-2000-CT LUNG SCREEN-20302/2-65646/175.xml
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0934/01-01-2000-CT LUNG SCREEN-69179/75647/171.xml
reading total...487/487 patient DICOMs
Worker 4: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0936/01-01-2000-CT GUIDED LUNG B-34718/2-15169/
reading total...250/259 patient DICOMs
reading total...259/259 patient DICOMs
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0936/01-01-2000-CT GUIDED LUNG B-34718/2-15169/169.xml
Worker 2: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0938/01-01-2000-CT GUIDED LUNG B-79121/2-32320/
reading tota

reading total...50/125 patient DICOMs
reading total...450/463 patient DICOMs
reading total...250/490 patient DICOMs
reading total...463/463 patient DICOMs
Worker 1: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0967/01-01-2000-CT LUNG SCREEN-66502/60723/
reading total...50/154 patient DICOMs
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0967/01-01-2000-CT LUNG SCREEN-66502/60723/138.xml
reading total...100/125 patient DICOMs
reading total...300/490 patient DICOMs
reading total...125/125 patient DICOMs
Worker 3: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0969/01-01-2000-CT LUNG SCREEN-46518/38194/
reading total...100/154 patient DICOMs
reading total...50/125 patient DICOMs
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0969/01-01-2000-CT LUNG SCREEN-46518/38194/136.xml
reading total...350/490 patient DICOMs
reading total...150/154 patient DICOMs
reading total...154/154 patient DICOMs
Worker 2: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-0970/01-01-2000-CHEST-22947/31474-38096/
reading total...100/125 patient DICOMs
reading total...

reading total...150/250 patient DICOMs
reading total...50/529 patient DICOMs
reading total...300/545 patient DICOMs
reading total...50/125 patient DICOMs
reading total...100/529 patient DICOMs
reading total...200/250 patient DICOMs
reading total...350/545 patient DICOMs
reading total...100/125 patient DICOMs
reading total...125/125 patient DICOMs
Worker 1: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-1010/01-01-2000-CT ANGIO CHEST NON-CO-81361/2-BOTTOM TO TOP-17588/
reading total...150/529 patient DICOMs
reading total...250/250 patient DICOMs
Worker 3: C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-1011/01-01-2000-CT THORAX WCONTRAST-94040/2-CHEST-06815/
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-1010/01-01-2000-CT ANGIO CHEST NON-CO-81361/2-BOTTOM TO TOP-17588/091.xml
reading total...400/545 patient DICOMs
C:/ntpu_project/LIDC-IDRI/LIDC-IDRI-1011/01-01-2000-CT THORAX WCONTRAST-94040/2-CHEST-06815/087.xml
reading total...200/529 patient DICOMs
reading total...50/252 patient DICOMs
reading total...450/545 patient

In [5]:
len(workflow.mapped_ct_dicom_list)

11645

In [6]:
workflow.mapped_ct_dicom_list[1].nodule_list

[<Nodule(image_uid=1.3.6.1.4.1.14519.5.2.1.6279.6001.314855836163512457174075547165, z_position=-176.500000 , type=NODULE_GREATER_THAN_3MM, inclusion=TRUE, len(edge_map_list)=14)>,
 <Nodule(image_uid=1.3.6.1.4.1.14519.5.2.1.6279.6001.314855836163512457174075547165, z_position=-176.5, type=NODULE_GREATER_THAN_3MM, inclusion=TRUE, len(edge_map_list)=23)>,
 <Nodule(image_uid=1.3.6.1.4.1.14519.5.2.1.6279.6001.314855836163512457174075547165, z_position=-176.5, type=NODULE_GREATER_THAN_3MM, inclusion=TRUE, len(edge_map_list)=23)>,
 <Nodule(image_uid=1.3.6.1.4.1.14519.5.2.1.6279.6001.314855836163512457174075547165, z_position=-176.5, type=NODULE_GREATER_THAN_3MM, inclusion=TRUE, len(edge_map_list)=26)>]

In [9]:
same_count = 0
for dicom in workflow.mapped_ct_dicom_list:
    same_count += dicom.is_same_exam_result
print(same_count)

8054
